In [ ]:
%%capture
!pip install simpletransformers
!pip install num2words
!pip install word2number
!pip install transformers[torch]


In [ ]:
import random
import warnings
import numpy as np
import logging
import os
import pandas as pd
from datetime import datetime
from num2words import num2words
from word2number import w2n

from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs


In [ ]:
#Connect to Google Drive
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer

checkpoint = "ramsrigouthamg/t5_paraphraser"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
!pip install evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 616.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=f33af64210a993beec6c8f80cf11454e83361c07ed588fa273c36693f7ace9d3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
model1 = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/w266/t5model')

In [ ]:
import torch
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
model2 = model1.to(torch_device)

In [ ]:
#make directory to store paraphrase outputs
#os.mkdir('/content/drive/MyDrive/w266/outputs_para_t5')

In [ ]:
predSummPath = '/content/drive/MyDrive/w266/outputs_para_t5'

documentPath = '/content/drive/MyDrive/w266/supert/outputs_ext'
testFiles = [file for file in os.listdir(documentPath)]
for file in sorted(testFiles):
  if os.stat(f'{documentPath}/{file}').st_size == 0:
    continue
  print(file)

  doc_in = open(f'{documentPath}/{file}', 'r', encoding='utf8')
  doc_lines = [line.strip() for line in doc_in.readlines()]
  pred_summary = []
  for line in doc_lines:
    text_to_paraphrase = line.strip().lower()
    #print(text_to_paraphrase)
    batch = tokenizer([text_to_paraphrase],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
    translated = model2.generate(**batch,max_length=60,num_beams=5, num_return_sequences=1, temperature=1.5)
    pred_line = tokenizer.batch_decode(translated, skip_special_tokens=True)
    pred_line = ''.join(pred_line)
    #print(pred_line)
    pred_summary.append(pred_line)

  with open(f'{predSummPath}/{file}', 'w') as predSumm_out:
    predSumm_out.write('\n'.join(pred_summary) + '\n')

2016-Apr-19-INTC_ext_summary.txt
2016-Apr-20-ASML_ext_summary.txt
2016-Apr-21-AMD_ext_summary.txt
2016-Apr-21-GOOGL_ext_summary.txt
2016-Apr-21-MSFT_ext_summary.txt
2016-Apr-26-AAPL_ext_summary.txt
2016-Apr-28-AMZN_ext_summary.txt
2016-Aug-11-NVDA_ext_summary.txt
2016-Aug-17-CSCO_ext_summary.txt
2016-Dec-21-MU_ext_summary.txt
2016-Feb-01-GOOGL_ext_summary.txt
2016-Feb-10-CSCO_ext_summary.txt
2016-Feb-17-NVDA_ext_summary.txt
2016-Jan-14-INTC_ext_summary.txt
2016-Jan-19-AMD_ext_summary.txt
2016-Jan-20-ASML_ext_summary.txt
2016-Jan-26-AAPL_ext_summary.txt
2016-Jan-28-AMZN_ext_summary.txt
2016-Jan-28-MSFT_ext_summary.txt
2016-Jul-19-MSFT_ext_summary.txt
2016-Jul-20-ASML_ext_summary.txt
2016-Jul-20-INTC_ext_summary.txt
2016-Jul-21-AMD_ext_summary.txt
2016-Jul-26-AAPL_ext_summary.txt
2016-Jul-28-AMZN_ext_summary.txt
2016-Jul-28-GOOGL_ext_summary.txt
2016-Jun-30-MU_ext_summary.txt
2016-Mar-30-MU_ext_summary.txt
2016-May-12-NVDA_ext_summary.txt
2016-May-18-CSCO_ext_summary.txt
2016-Nov-10-NVDA

In [ ]:
def getPredData(dataPath):
  predPath = f'dataPath'
  testFiles = [file for file in os.listdir(dataPath)]
  PredData = []
  for file in sorted(testFiles):
    dataset = []
    doc_in = open(f'{dataPath}/{file}', 'r', encoding='utf8')
    doc_lines = [line.strip() for line in doc_in.readlines()]
    for line in doc_lines:
      text_to_paraphrase = line.strip().lower()
      dataset.append(text_to_paraphrase)
    PredData.append('\n'.join(dataset) + '\n')

  return PredData

In [ ]:
dataPathPred = '/content/drive/MyDrive/w266/outputs_para_t5'

In [ ]:
PredParaDataset = getPredData(dataPathPred)

In [ ]:
PredParaDataset[0]

"sees fy 2021 as an opportunity to reinvest in the business and accelerate growth.\nsees fy 2021 mid-single-digit growth.\ni'm trying to see if you can be just a bit more specific on your full-year view. does the team anticipate double-digits growth in dcg?\niot grew 22% in quarter.\nsees fy 2021 earnings per share $800 million.\nreaffirms that it's time to try again, and that includes pc, but it's much more than that.\nif you look at cfo commentary, just to make sure, $1.4 billion in run rate savings is a net number.\nsees q2 total expenses will drop about $150 million a quarter from 4q 2016 to q2 of 2017?\nccg operating profit is up over 30%.\ni've given you this range of 55% to 65% gross margin as being -- most of the data points landing between those goal posts.\ncfo - if you look at the cfo commentary, we actually gave you desktop and notebook. what you'll see is notebook was pretty flat from an asp standpoint year-on-year. desktop was up 6%, so that's one of\ni know you probably 

In [ ]:
refPath = '/content/drive/MyDrive/w266/supert/ref_summaries'

In [ ]:
refParaDataset = getPredData(refPath)

In [ ]:
refParaDataset[0]

"hello.\nsure.\nand so what this effort around the restructuring is, is to say, it's time now to try, and we've made enough progress, right?\nand the one that stacy and i have been working for several months, actually several quarters, on what does he do next, how do we growth his both exposure to other parts of the company and also the rest of the company see his leadership style.\nthis is going to be an orderly transition.\nmy follow-up would be the full-year guidance of mid single-digit growth.\nwhat we're projecting for the year is overall growth, and we think we'll see growth in the data center, growth in iot, growth in memory that's offsetting some, oh and of course we've added in altera now into our product family, and so you take that and that's offsetting some of the weakness that we're seeing in the pc market.\ngreat, thank you.\nyou know, we think that we can become more focused.\nyou did a great job explaining exactly why we believe that the data center will continue in dou

In [ ]:
def getScores(dataset1, dataset2):
  scores = []
  for i in range(len(dataset1)):
    scores1 = rouge.compute(
        predictions=[dataset1[i]], references=[dataset2[i]]
        )
    scores.append(scores1)
  rouge1 = []
  rouge2 = []
  rougeL = []
  rougeLsum = []
  for i in range(len(scores)):
    rouge1.append(scores[i]['rouge1'])
    rouge2.append(scores[i]['rouge2'])
    rougeL.append(scores[i]['rougeL'])
    rougeLsum.append(scores[i]['rougeLsum'])

  rouge1Avg = sum(rouge1) / len(rouge1)
  rouge2Avg = sum(rouge2) / len(rouge2)
  rougeLAvg = sum(rougeL) / len(rougeL)
  rougeLsumAvg = sum(rougeLsum) / len(rougeLsum)
  return 'rouge 1 Avg', rouge1Avg, 'rouge 2 Avg', rouge2Avg, 'rouge L Avg', rougeLAvg, 'rouge LSum Avg', rougeLsumAvg

In [ ]:
T5Scores = getScores(PredParaDataset,refParaDataset)

In [ ]:
T5Scores

('rouge 1 Avg',
 0.38422677842564,
 'rouge 2 Avg',
 0.12829878658948057,
 'rouge L Avg',
 0.15359547754526456,
 'rouge LSum Avg',
 0.3628985662542239)

In [ ]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.6 MB/s eta 0:00:00


In [ ]:
from evaluate import load
bertscore = load("bertscore")
results = bertscore.compute(predictions=PredParaDataset, references=refParaDataset, lang="en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print('The average precision is', sum(results['precision'])/len(results['precision']), '\nThe average recall is', sum(results['recall'])/len(results['recall']), '\nThe average F1 score is', sum(results['f1'])/len(results['f1']))

The average precision is 0.8380207484895057 
The average recall is 0.8369528782236707 
The average F1 score is 0.837433180638722
